In [1]:
from imblearn.under_sampling import NearMiss
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import numpy as np

In [2]:
data_regression = pd.read_csv("energy_task.csv")
data_classification = pd.read_csv("neo_task.csv")

In [3]:
data_regression=data_regression.dropna()
data_classification=data_classification.dropna()
data_regression = data_regression.drop(["date"], axis=1)
data_classification = data_classification.drop(["name"], axis=1)
data_classification = data_classification.drop(["id"], axis=1)

In [4]:
dict1 = {'True': '1', 'False': '0'}

data_classification['hazardous']=data_classification['hazardous'].astype(str)
data_classification.loc[:, 'hazardous'] = data_classification['hazardous'].map(dict1)
data_classification['hazardous']=data_classification['hazardous'].astype(int)

In [5]:
data_regression.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9


In [6]:
data_classification.head(10)

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,0.016016,0.035813,56014.078517,1.024333e+06,26.1,0
1,0.030518,0.068240,7864.348060,3.268186e+07,24.7,0
2,0.055533,0.124177,55257.544508,6.538636e+07,23.4,0
3,0.019256,0.043057,41531.404722,1.260796e+07,25.7,0
4,0.139494,0.311918,67639.394481,7.130590e+07,21.4,0
5,0.046191,0.103286,18933.875801,2.847083e+07,23.8,0
6,0.116026,0.259442,43184.402732,3.418417e+07,21.8,0
7,0.021113,0.047211,38064.802287,2.653154e+07,25.5,0
8,0.221083,0.494356,33736.859964,5.670662e+07,20.4,0
9,0.242412,0.542051,56188.215588,6.175441e+07,20.2,0


In [7]:

y_regression = data_regression["Appliances"]
X_regression = data_regression.drop(columns = ['Appliances'])
y_classification = data_classification['hazardous']
X_classification = data_classification.drop(columns = ['hazardous'])


In [8]:
from sklearn.model_selection import train_test_split
X_regression_train, X_regression_test, y_regression_train, y_regression_test = train_test_split(X_regression,
                                                                                                y_regression,
                                                                                                test_size=0.2)
X_classification_train, X_classification_test, y_classification_train, y_classification_test = train_test_split(X_classification,
                                                                                                                y_classification,
                                                                                                                test_size=0.2,
                                                                                                               stratify=y_classification)

In [9]:
X_classification

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude
0,0.016016,0.035813,56014.078517,1.024333e+06,26.10
1,0.030518,0.068240,7864.348060,3.268186e+07,24.70
2,0.055533,0.124177,55257.544508,6.538636e+07,23.40
3,0.019256,0.043057,41531.404722,1.260796e+07,25.70
4,0.139494,0.311918,67639.394481,7.130590e+07,21.40
...,...,...,...,...,...
90831,0.017561,0.039268,23264.740825,1.635007e+06,25.90
90832,0.110804,0.247765,24802.519406,3.351901e+07,21.90
90833,0.035039,0.078350,116288.999548,5.471396e+07,24.40
90834,0.044112,0.098637,45763.317060,2.694877e+07,23.90


In [10]:

y_classification

0        0
1        0
2        0
3        0
4        0
        ..
90831    0
90832    0
90833    0
90834    0
90835    0
Name: hazardous, Length: 88540, dtype: int32

# Регрессия :

In [26]:

# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(25,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [27]:
# посмотрим, какая сеть у нас получилась
model_regression.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                1664      
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 4289 (16.75 KB)
Trainable params: 4289 (

In [28]:
# компилируем
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")

In [29]:
# обучаем, 10 эпох означает 10 проходов по обучающей выборке
model_regression.fit(X_regression_train, y_regression_train, epochs=10)

Epoch 1/10
464/464 [==============================] - 3s 4ms/step - loss: 11212.4316
Epoch 2/10
464/464 [==============================] - 2s 4ms/step - loss: 10602.1885
Epoch 3/10
464/464 [==============================] - 2s 4ms/step - loss: 10420.6328
Epoch 4/10
464/464 [==============================] - 2s 4ms/step - loss: 10342.2002
Epoch 5/10
464/464 [==============================] - 2s 4ms/step - loss: 10258.7598
Epoch 6/10
464/464 [==============================] - 2s 4ms/step - loss: 10250.8389
Epoch 7/10
464/464 [==============================] - 2s 4ms/step - loss: 10200.9395
Epoch 8/10
464/464 [==============================] - 2s 4ms/step - loss: 10145.3301
Epoch 9/10
464/464 [==============================] - 2s 4ms/step - loss: 10136.1973
Epoch 10/10
464/464 [==============================] - 2s 4ms/step - loss: 10131.3369


In [30]:
# оцениваем качество с помощью метрик
print(mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test)))
print(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test)))

116/116 [==============================] - 0s 3ms/step
48.688613951270504
116/116 [==============================] - 0s 3ms/step
8853.887416199115


# Классификация :

In [16]:
w0 = 1 / y_classification_train[y_classification_train==0].shape[0]
w1 = 1 / y_classification_train[y_classification_train==1].shape[0]


In [17]:
total_samples = len(y_classification_train)
class_weight = {0: total_samples / (2 * np.sum(y_classification_train == 0)),
                1: total_samples / (2 * np.sum(y_classification_train == 1))}

In [31]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(32, activation="relu", input_shape=(5,)),
        tf.keras.layers.Dense(16, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy")
model_classification_1.fit(X_classification_train, y_classification_train, epochs=10, verbose=None,
                           class_weight={0: w0, 1: w1})
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     15989
           1       0.10      1.00      0.18      1719

    accuracy                           0.10     17708
   macro avg       0.05      0.50      0.09     17708
weighted avg       0.01      0.10      0.02     17708

[[    0 15989]
 [    0  1719]]


C:\Osnova\Programming\Python\ML\test\venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Osnova\Programming\Python\ML\test\venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Osnova\Programming\Python\ML\test\venv\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
model_classification_1.save('Neural.h5')

C:\Osnova\Programming\Python\ML\test\venv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
class MLP:
    def __init__(self, layers, activation_functions):
        self.layers = layers
        self.activation_functions = activation_functions
        self.weights = [np.random.rand(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]
        self.activations = []
        self.derivatives = []

    def initialize_weights(self, input_size):
        self.weights[0] = np.random.rand(input_size, self.layers[1])
        for i in range(1, len(self.layers)-1):
            self.weights[i] = np.random.rand(self.layers[i], self.layers[i+1])

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)

    def linear(self, x):
        return x

    def linear_derivative(self, x):
        return np.ones_like(x)

    def forward(self, X):
        self.activations = []
        self.derivatives = []
        layer_output = X
        self.activations.append(layer_output)

        for i in range(len(self.layers) - 1):
            weighted_sum = np.dot(layer_output, self.weights[i]) + self.biases[i]
            if self.activation_functions[i] == 'sigmoid':
                layer_output = self.sigmoid(weighted_sum)
                self.derivatives.append(self.sigmoid_derivative(layer_output))
            elif self.activation_functions[i] == 'relu':
                layer_output = self.relu(weighted_sum)
                self.derivatives.append(self.relu_derivative(weighted_sum))
            elif self.activation_functions[i] == 'linear':
                layer_output = self.linear(weighted_sum)
                self.derivatives.append(self.linear_derivative(weighted_sum))

            self.activations.append(layer_output)

        return layer_output

    def backward(self, X, y, learning_rate):
        errors = y - self.activations[-1]
        for i in reversed(range(len(self.layers)-1)):
            delta = errors * self.derivatives[i]
            self.weights[i] += learning_rate * np.dot(self.activations[i].T, delta)
            self.biases[i] += learning_rate * np.sum(delta, axis=0, keepdims=True)
            errors = delta.dot(self.weights[i].T)

    def mean_squared_error(self, y_true, y_pred):
        return np.mean((y_true - y_pred)**2)

    def accuracy(self, y_true, y_pred):
        y_pred_binary = np.round(y_pred)
        return np.mean(y_true == y_pred_binary)

    def train(self, X, y, epochs, learning_rate, type):
        input_size = X.shape[1]
        self.initialize_weights(input_size)

        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y, learning_rate)
            
            if epoch % 100 == 0 and type == 'reg':
                mse = self.mean_squared_error(y, self.activations[-1])
                print(f"Epoch {epoch}, Mean Squared Error: {mse}")

            elif epoch % 100 == 0 and type == 'clf':
                acc = self.accuracy(y, self.activations[-1])
                print(f"Epoch {epoch}, Accuracy: {acc}")

In [24]:
X_r = X_regression.values
y_r = y_regression.values.reshape(-1, 1)

input_size = 4
mlp_reg = MLP(layers=[input_size, 5, 1], activation_functions=['relu', 'relu'])
mlp_reg.initialize_weights(input_size)
mlp_reg.train(X_r, y_r, epochs=1000, learning_rate=0.01, type='reg')


Epoch 0, Mean Squared Error: 4279251.966770145
Epoch 100, Mean Squared Error: 20052.368108749597
Epoch 200, Mean Squared Error: 20052.368108749597
Epoch 300, Mean Squared Error: 20052.368108749597
Epoch 400, Mean Squared Error: 20052.368108749597
Epoch 500, Mean Squared Error: 20052.368108749597
Epoch 600, Mean Squared Error: 20052.368108749597
Epoch 700, Mean Squared Error: 20052.368108749597
Epoch 800, Mean Squared Error: 20052.368108749597
Epoch 900, Mean Squared Error: 20052.368108749597


In [25]:
X_c = X_classification.values
y_c = y_classification.values.reshape(-1, 1)

input_size = 4
mlp_cls = MLP(layers=[input_size, 5, 1], activation_functions=['relu', 'sigmoid'])
mlp_cls.initialize_weights(input_size)
mlp_cls.train(X_c, y_c, epochs=1000, learning_rate=0.001, type='clf')

Epoch 0, Accuracy: 0.09708606279647616
Epoch 100, Accuracy: 0.09708606279647616
Epoch 200, Accuracy: 0.09708606279647616
Epoch 300, Accuracy: 0.09708606279647616
Epoch 400, Accuracy: 0.09708606279647616
Epoch 500, Accuracy: 0.09708606279647616
Epoch 600, Accuracy: 0.09708606279647616
Epoch 700, Accuracy: 0.09708606279647616
Epoch 800, Accuracy: 0.09708606279647616
Epoch 900, Accuracy: 0.09708606279647616


# MLP